In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder, normalize
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib
from mlxtend.plotting import plot_confusion_matrix
import warnings
from scipy.signal import savgol_filter, find_peaks, correlate
from sklearn.decomposition import PCA
import scipy.fftpack as FFT
from sklearn.utils import shuffle
import joblib
warnings.filterwarnings('ignore')
matplotlib.rcParams['figure.figsize'] = [12,5]

## help functions

In [2]:
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

def convert_to_datetime(x):
    dt = datetime.fromtimestamp(x // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(x % 1000000000)).zfill(9)
    return s

def heatmap(cm, labels):
    df_cm = pd.DataFrame(cm, index=labels, columns=labels)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)

    plt.show()
    
def preprocess(data, activity, drop_interval_begin = 3, drop_interval_end = 3):
    #convert nanoseconds to date 
    data['time'] = data['time'].apply(convert_to_datetime) 
    data['time'] = pd.to_datetime(data['time'])

#convert to same data type
    for i in range(1, 4): 
        data[column_names_org[i]] = data[column_names_org[i]].apply(convert_to_float)
    
#drop rows with NaN values 
    data.dropna(axis=0, how='any', inplace=True) #TODO: invullen met mean, mod of median / interpolatie

#drop duplicates 
    data.drop_duplicates(subset=None, keep='first', inplace = True)
    
    #drop first and last 3 sec
    indexFirst = data[ (data['time'].iloc[0]+ pd.to_timedelta(drop_interval_begin, unit='s')) > data['time'] ].index
    data.drop(indexFirst , inplace=True)
    indexLast = data[ (data['time'].iloc[-1]- pd.to_timedelta(drop_interval_end, unit='s')) < data['time'] ].index
    data.drop(indexLast , inplace=True)
    
    #add activity label
    data['activity'] = activity
    
    return data

#feature extraction
def get_mean_window(df):
    return df.mean()

def get_min_window(df):
    return df.min()

def get_max_window(df):
    return df.max()

def get_std_window(df):
    return df.std()

def get_med_window(df):
    return df.median()

#TODO: scipy integrate
#TODO: voor elke as apart??
#measure of activity level (m/s²)
def get_signal_magnitude_area(df):
        sum = 0
        for i in range(0, len(df)):
            sum += (abs(df['x'].iloc[i]) + abs(df['y'].iloc[i]) + abs(df['z'].iloc[i]))
        return sum /len(df)
    #result = integrate.quad(lambda t: df['x'].apply(lambda n : abs(n)) + df['y'].apply(lambda n : abs(n)) + df['z'].apply(lambda n : abs(n)), 0, len(df))

#TODO: voor elke as apart??
#degree of movement intensity (m/s²)
def get_signal_magnitude_vector(df):
    sum = 0
    for i in range(0, len(df)):
        sum += math.sqrt(df['x'].iloc[i] * df['x'].iloc[i] + df['y'].iloc[i] * df['y'].iloc[i] + df['z'].iloc[i] * df['z'].iloc[i])
    return sum

#average angle (radian) between accelerometer vector and x as (parallel with arm)
def get_tilt_angle(df):
    df_cos = pd.DataFrame(columns=["tilt_ang"])
    df_dot = df['x']
    for i in range(0, len(df)):
        s = pd.Series({"tilt_ang" : (df_dot.iloc[i])/(math.sqrt(df['x'].iloc[i]*df['x'].iloc[i] + df['y'].iloc[i]*df['y'].iloc[i] + df['z'].iloc[i]*df['z'].iloc[i]))})
        df_cos=df_cos.append(s, ignore_index=True)
    df_angle = np.arccos(df_cos)
    return df_angle.mean()['tilt_ang']

def get_power_spectral_density(df):
    df_psd = np.abs(df)**2
    return df_psd.sum()

#TODO: datatype is object en niet compex nr
def get_entropy(df):
    entropy = []
    pdf = df / df.sum()
    for i in range (1, len(pdf.columns)):
        entropy.append(np.complex(-np.nansum(pdf.iloc[:,i] * np.log2(pdf.iloc[:,i]))))
    return entropy

#generate windows with 50% overlap
def windows(df, time, overlap):
    start = df.iloc[0]
    while  (start + pd.to_timedelta(time, unit='s')) < df.iloc[-1]:
        yield start, (start + pd.to_timedelta(time, unit='s'))
        if overlap:
            start += pd.to_timedelta(time/2, unit='s')
        else:
            start += pd.to_timedelta(time, unit='s')
    #last samples 
    yield (df.iloc[-1] - pd.to_timedelta(time, unit='s')), df.iloc[-1]
        
def feature_extraction_segmentation(data, window, overlap):
    column_names = ["x_mean", "y_mean", "z_mean", "x_min", "y_min", "z_min", "x_max", "y_max", "z_max",
                "x_std", "y_std", "z_std", "x_med", "y_med", "z_med", "activity"]
    df = pd.DataFrame(columns = column_names)

    for (start, end) in windows(data['time'], window, overlap):
        vw1 = data['time'] >= start
        vw2 = data['time'] < end
        mean = get_mean_window(data[vw1 & vw2])
        min = get_min_window(data[vw1 & vw2])
        max = get_max_window(data[vw1 & vw2])
        std = get_std_window(data[vw1 & vw2])
        med = get_med_window(data[vw1 & vw2])
        sma = get_signal_magnitude_area(data[vw1 & vw2])
        smv = get_signal_magnitude_vector(data[vw1 & vw2])
        tilt = get_tilt_angle(data[vw1 & vw2])
        #fourrier transform
        t_x = data[vw1 & vw2][['time','x']].set_index('time')
        t_y = data[vw1 & vw2][['time','y']].set_index('time')
        t_z = data[vw1 & vw2][['time', 'z']].set_index('time')

        df_f = pd.DataFrame(columns=['f', 'x_f', 'y_f', 'z_f'])

    #TODO: determine sampling rate (datapoints per second)
        sampling_rate = 52

        df_f['x_f'] = FFT.fft(t_x).ravel()
        df_f['y_f'] = FFT.fft(t_y).ravel()
        df_f['z_f'] = FFT.fft(t_z).ravel()
        df_f['f'] = FFT.fftfreq(len(df_f['x_f'])) * sampling_rate

        psd = get_power_spectral_density(df_f)
        #entropy = get_entropy(df_f)

        df = df.append(pd.Series({'x_mean': mean['x'], 'y_mean': mean['y'], 'z_mean': mean['z'], "x_min" : min['x'],
                              "y_min" : min['y'], "z_min" : min['z'], "x_max" : max['x'], "y_max" : max['y'], "z_max" : max['z'],
                              "x_std" : std['x'], "y_std" : std['y'], "z_std" : std['z'], "x_med" : med['x'], "y_med" : med['y'], "z_med" : med['z'],
                              "sma" : sma, "smv" : smv, "tilt" : tilt, "x_psd" : psd['x_f'], "y_psd" : psd['y_f'], "z_psd" : psd['z_f']
                              , "activity" : data['activity'].iloc[0]}), ignore_index=True) # "x_entropy" : entropy[0], "y_entropy" : entropy[1], "z_entropy" : entropy[2],
    return df

#3 keer savgol filter uitvoeren met 33,5
def get_turns_smooth_side_swing(df):
    for i in range(0,5):
        df['x'] = savgol_filter(df[ 'x'].to_numpy(), 101, 5)
        df['y'] = savgol_filter(df[ 'y'].to_numpy(), 101, 5)
        df['z'] = savgol_filter(df[ 'z'].to_numpy(), 101, 5)
        df.plot(x='time', subplots=True)

    wx = find_peaks(df['x'])
    wy = find_peaks(df['y'])
    wz = find_peaks(df['z'])
    return (len(wx[0]) + len(wy[0]) + len(wz[0]))/3


column_names_org = ['time', 'x', 'y', 'z']

# <font color='green' >PROEFPERSOON 1

## mistake

In [230]:
mistake1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_left_forward-tim.csv", sep=';', header=0, skipinitialspace=True)
mistake2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_tim(meting2).csv", sep=';', header=0, skipinitialspace=True)
mistake3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_left_backwards_tim1.csv", sep=';', header=0, skipinitialspace=True)
mistake4 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_left_backwards_tim2.csv", sep=';', header=0, skipinitialspace=True)
mistake5 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_left_backwards_tim3.csv", sep=';', header=0, skipinitialspace=True)
mistake6 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_right_backwards_tim.csv", sep=';', header=0, skipinitialspace=True)
mistake7 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_right_backwards_tim2.csv", sep=';', header=0, skipinitialspace=True)
mistake8 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon1\jump_fast_right_backwards_tim3.csv", sep=';', header=0, skipinitialspace=True)

mistake9 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\forward_180\proefpersoon1\forward_180_tim.csv", sep=';', header=0, skipinitialspace=True)
mistake10 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\forward_180\proefpersoon1\forward_180_left_forward_tim.csv", sep=';', header=0, skipinitialspace=True)

mistake11 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_right_backwards_tim.csv", sep=';', header=0, skipinitialspace=True)
mistake12 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_right_backwards_tim1.csv", sep=';', header=0, skipinitialspace=True)
mistake13 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_backwards_tim1.csv", sep=';', header=0, skipinitialspace=True)
mistake14 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_backwards_tim2.csv", sep=';', header=0, skipinitialspace=True)
mistake15 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_backwards_tim3.csv", sep=';', header=0, skipinitialspace=True)
mistake16 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_tim.csv", sep=';', header=0, skipinitialspace=True)
mistake17 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon1\jump_slow_left_forward_tim.csv", sep=';', header=0, skipinitialspace=True)

In [231]:
mistake1 = preprocess(mistake1, "mistake", 1,1)
mistake3 = preprocess(mistake3, "mistake", 1,1)
mistake6 = preprocess(mistake6, "mistake", 1,1)
mistake13 = preprocess(mistake13, "mistake", 1,1)
mistake14 = preprocess(mistake14, "mistake", 1,1)
mistake15 = preprocess(mistake15, "mistake", 1,1)
mistake16 = preprocess(mistake16, "mistake", 1,1)
mistake17 = preprocess(mistake17, "mistake", 1,1)

In [232]:
cut1 = mistake1[mistake1["time"] >= pd.to_datetime("1970-01-01 01:08:57.0000")]
cut2 = mistake3[mistake3["time"] >= pd.to_datetime("1970-01-02 01:02:04.9000")]
cut3 = mistake6[mistake6["time"] >= pd.to_datetime("1970-01-01 21:18:04.9100")]
cut4 = mistake13[mistake13["time"] >= pd.to_datetime("1970-01-02 00:59:40.0000")]
cut5 = mistake14[mistake14["time"] >= pd.to_datetime("1970-01-02 01:00:45.7000")]
cut6 = mistake15[mistake15["time"] >= pd.to_datetime("1970-01-02 01:01:30.6000")]
cut7 = mistake16[mistake16["time"] >= pd.to_datetime("1970-01-01 11:40:06.9000")]
cut2

,time,x,y,z,activity
372,1970-01-02 01:02:04.909584384,-3.617948,-3.876373,2.019542,mistake
373,1970-01-02 01:02:04.934750208,-2.622534,-1.512264,1.557728,mistake
374,1970-01-02 01:02:04.951527424,-2.378466,-2.450250,-1.610370,mistake
375,1970-01-02 01:02:04.968304640,-1.899901,-5.852844,-2.471786,mistake
376,1970-01-02 01:02:04.993470464,-0.775275,-4.007978,-3.251846,mistake
...,...,...,...,...,...
442,1970-01-02 01:02:06.318870528,-3.625126,1.093520,9.391829,mistake
443,1970-01-02 01:02:06.318870528,-3.639483,1.184447,9.485149,mistake
444,1970-01-02 01:02:06.335647744,-3.639483,1.184447,9.485149,mistake
445,1970-01-02 01:02:06.360813568,-3.639483,1.184447,9.485149,mistake


niet gewoon achter elkaar plakken, want is niet realistisch

### SEGMENTATION

In [233]:
cut1 = feature_extraction_segmentation(cut1,1, True)
cut2 = feature_extraction_segmentation(cut2,1, True)
cut3 = feature_extraction_segmentation(cut3,1, True)
cut4 = feature_extraction_segmentation(cut4,1, True)
cut5 = feature_extraction_segmentation(cut5,1, True)
cut6 = feature_extraction_segmentation(cut6,1, True)
cut7 = feature_extraction_segmentation(cut7,1, True)

In [234]:
mistakes_1 = pd.concat([cut1, cut2, cut3, cut4, cut5, cut6, cut7],  ignore_index=True)
mistakes_1.shape

(18, 22)

# <font color='green' >PROEFPERSOON 2

## mistakes

In [299]:
forward_180 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\forward_180\proefpersoon2\forward_180_2.csv", sep=';', header=0, skipinitialspace=True)

cross_over_right_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\cross_over_right_backwards1.csv", sep=';', header=0, skipinitialspace=True)
cross_over_right_backwards2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\cross_over_right_backwards2.csv", sep=';', header=0, skipinitialspace=True)
cross_over_right_backwards3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\cross_over_right_backwards3.csv", sep=';', header=0, skipinitialspace=True)
cross_over_right_forward1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossOver_right_forward1.csv", sep=';', header=0, skipinitialspace=True)
cross_over_right_forward2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossOver_right_forward2.csv", sep=';', header=0, skipinitialspace=True)
cross_over_right_forward3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossOver_right_forward3.csv", sep=';', header=0, skipinitialspace=True)
cross_over_left_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossover_backwards.csv", sep=';', header=0, skipinitialspace=True)
cross_over_left_backwards2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossover_backwards(2).csv", sep=';', header=0, skipinitialspace=True)
cross_over_left_backwards3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossOver_left_backwards.csv", sep=';', header=0, skipinitialspace=True)
cross_over_left_backwards4 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossOver_left_backwards2.csv", sep=';', header=0, skipinitialspace=True)
cross_over_left_backwards5 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\crossOver_left_backwards3.csv", sep=';', header=0, skipinitialspace=True)
cross_over2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\cross_over2.csv", sep=';', header=0, skipinitialspace=True)
cross_over3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\cross_over3.csv", sep=';', header=0, skipinitialspace=True)
cross_over4 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\cross_over4.csv", sep=';', header=0, skipinitialspace=True)
cross_over5 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\cross_over\proefpersoon2\cross_over5.csv", sep=';', header=0, skipinitialspace=True)

jump_slow_right_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right_backwards1.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_right_backwards2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right_backwards2.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_turn_diff = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_turn_diff.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_rigth = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right1.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_rigth2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow_right2.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_rigth3 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jumpslow_right_forward.csv", sep=';', header=0, skipinitialspace=True)
jump_slow_rigth4 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jumpslow_right_forward2.csv", sep=';', header=0, skipinitialspace=True)
jump_slow = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow.csv", sep=';', header=0, skipinitialspace=True)
jump_slow2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_slow\proefpersoon2\jump_slow1 (meting2).csv", sep=';', header=0, skipinitialspace=True)

jump_fast_right_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon2\jump_fast_right_backwards1.csv", sep=';', header=0, skipinitialspace=True)
jump_fast_right_backwards2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon2\jump_fast_right_backwards2.csv", sep=';', header=0, skipinitialspace=True)
jump_fast_left_backwards1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon2\jump_fast_backwards.csv", sep=';', header=0, skipinitialspace=True)
jump_fast_right_forward1 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon2\jump_fast_right_forward1.csv", sep=';', header=0, skipinitialspace=True)
jump_fast_right_forward2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon2\jump_fast_right_forward2.csv", sep=';', header=0, skipinitialspace=True)
jump_fast = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon2\jump_fast.csv", sep=';', header=0, skipinitialspace=True)
jump_fast2 = pd.read_csv(r"C:\Users\Elise\Documents\unif\master\semester2\masterproef\gitProject\thesis\data\jump_fast\proefpersoon2\jump_fast (meting2).csv", sep=';', header=0, skipinitialspace=True)

In [300]:
mistake1 = preprocess(cross_over_right_backwards1, "mistake",1,1)
mistake2 = preprocess(cross_over_right_backwards2, "mistake",1,1)
mistake3 = preprocess(cross_over_right_backwards3, "mistake",1,1)
mistake4 = preprocess(cross_over_right_forward1, "mistake",1,1)
mistake5 = preprocess(cross_over_right_forward2, "mistake",1,1)
mistake6 = preprocess(cross_over_left_backwards1, "mistake",1,1)
mistake7 = preprocess(cross_over_left_backwards2, "mistake",1,1)
mistake8 = preprocess(cross_over_left_backwards3, "mistake",1,1)
mistake9 = preprocess(cross_over_left_backwards5, "mistake",1,1)
mistake10 = preprocess(cross_over2, "mistake",1,1)
mistake11 = preprocess(jump_slow_right_backwards1, "mistake",1,1)
mistake12 = preprocess(jump_slow_turn_diff, "mistake",1,1)
mistake13 = preprocess(jump_fast_right_backwards1, "mistake",1,1)
mistake14 = preprocess(jump_fast_right_backwards2, "mistake",1,1)
mistake15 = preprocess(jump_fast_left_backwards1, "mistake",1,1)
mistake16 = preprocess(jump_fast_right_forward1, "mistake",1,1)
mistake17 = preprocess(jump_fast_right_forward2, "mistake",1,1)
mistake18 = preprocess(jump_fast, "mistake",1,1)
mistake19 = preprocess(jump_fast2, "mistake",1,1)
mistake19.head()

,time,x,y,z,activity
63,1970-01-01 04:38:00.990842880,-1.842473,-9.738789,-6.283552,mistake
64,1970-01-01 04:38:00.998182912,-1.864009,-9.934999,-6.257231,mistake
65,1970-01-01 04:38:01.004474368,-1.933401,-10.083355,-5.644669,mistake
66,1970-01-01 04:38:01.027543040,-1.962115,-10.408778,-5.668597,mistake
67,1970-01-01 04:38:01.041174528,-1.962115,-10.408778,-5.668597,mistake


In [301]:
cut1 = mistake1[mistake1['time'] >= pd.to_datetime("1970-01-02 01:08:20.7000")]
cut2 = mistake2[mistake2['time'] >= pd.to_datetime("1970-01-02 01:09:22.4000")]
cut3 = mistake3[mistake3['time'] >= pd.to_datetime("1970-01-02 01:10:10.8000")]
cut4 = mistake4[mistake4['time'] >= pd.to_datetime("1970-01-01 21:24:27.6000")]
cut5 = mistake5[mistake5['time'] >= pd.to_datetime("1970-01-01 21:24:54.3500")]
cut6 = mistake6[mistake6['time'] >= pd.to_datetime("1970-01-01 01:48:41.9500")]
cut7 = mistake7[mistake7['time'] >= pd.to_datetime("1970-01-01 01:49:13.0500")]
cut8 = mistake8[mistake8['time'] >= pd.to_datetime("1970-01-01 18:24:46.4500")] #kan mss in 2 gesplitst worden want piek ertussen, slechtere detectie??
cut9 = mistake9[mistake9['time'] >= pd.to_datetime("1970-01-01 18:26:20.2000")]
cut10 = mistake10[mistake10['time'] >= pd.to_datetime("1970-01-01 11:20:13.3500")]
cut11 = mistake11[mistake11['time'] >= pd.to_datetime("1970-01-02 01:12:21.5000")]
cut12 = mistake12[mistake12['time'] >= pd.to_datetime("1970-01-01 11:27:34.6000")]
cut13 = mistake13[mistake13['time'] >= pd.to_datetime("1970-01-02 01:06:08.1000")]
cut14 = mistake14[mistake14['time'] >= pd.to_datetime("1970-01-02 01:07:07.7000")]
cut15 = mistake15[mistake15['time'] >= pd.to_datetime("1970-01-01 01:46:25.9000")]
cut16 = mistake16[mistake16['time'] >= pd.to_datetime("1970-01-01 21:22:13.3000")]
cut17 = mistake17[mistake17['time'] >= pd.to_datetime("1970-01-01 21:23:18.2000")]   #piek
cut18 = mistake18[mistake18['time'] >= pd.to_datetime("1970-01-01 01:24:11.1200")]
cut19 = mistake19[mistake19['time'] >= pd.to_datetime("1970-01-01 04:38:42.5000")] 

### SEGMENTATION

In [302]:
cut1 = feature_extraction_segmentation(cut1,1, True)
cut2 = feature_extraction_segmentation(cut2,1, True)
cut3 = feature_extraction_segmentation(cut3,1, True)
cut4 = feature_extraction_segmentation(cut4,1, True)
cut5 = feature_extraction_segmentation(cut5,1, True)
cut6 = feature_extraction_segmentation(cut6,1, True)
cut7 = feature_extraction_segmentation(cut7,1, True)
cut8= feature_extraction_segmentation(cut8,1, True)
cut9= feature_extraction_segmentation(cut9,1, True)
cut10 = feature_extraction_segmentation(cut10,1, True)
cut11 = feature_extraction_segmentation(cut11,1, True)
cut12 = feature_extraction_segmentation(cut12,1, True)
cut13 = feature_extraction_segmentation(cut13, 1,True)
cut14 = feature_extraction_segmentation(cut14,1, True)
cut15 = feature_extraction_segmentation(cut15,1, True)
cut16 = feature_extraction_segmentation(cut16, 1,True)
cut17 = feature_extraction_segmentation(cut17,1, True)
cut18 = feature_extraction_segmentation(cut18, 1,True)
cut19 = feature_extraction_segmentation(cut19,1, True)

In [303]:
mistakes = pd.concat([cut1, cut2, cut3, cut4, cut5, cut6, cut7, cut8,cut9, cut10,cut11, cut12, cut13, cut14, cut15, cut16, cut17, cut18, cut19  ],  ignore_index=True)
mistakes.shape

(34, 22)